# ITMAL Exercise

REVISIONS| |
---------| |
2018-0318| CEF, initial.
2018-0321| CEF, synced with MLP moon exercise.
2018-0323| CEF, minor updated and spell checked.
2019-0930| CEF, updated for ITMAL E19.


## Keras Multi-Layer Perceptrons (MLP's) on MNIST-data


### Qa Using a Keras MLP on the MNIST-data

Now, make a Keras `Sequential` model and fit it to the MNIST data, re-using as much of the code form the `mlp_moon.ipynb` as you can.

Then try to change the number of hidden layers and the neurons in each layer, looking for increases in test accuracy via ``score``. 

Publish your best score for your model in Blackboard, see link under L06. We use categorical accuracy for score---eventhough a $F_1$ score could say more. Publish you result like
```
   ITMALGrpXY: score=0.76, a 10-20-30-20-10 MLP, takes looong to train
```
or similar


NOTE: you probably need to scale/normalize the MNIST data before a fit, and no 2D-decision boundaries can be drawn from the 784-dimension MNIST data.

### Import and normalize data from scikit-learn

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.datasets import fetch_openml
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam

np.random.seed(42)

# Get MNIST data:
X, y = fetch_openml('mnist_784', cache=True, return_X_y=True)
X = X.astype('float32') / 255.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



y_train_binary = to_categorical(y_train)
y_test_binary  = to_categorical(y_test)

assert y.ndim==1
assert y_train_binary.ndim==2
assert y_test_binary.ndim ==2

Using TensorFlow backend.


### Build original model from moon notebook

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build Keras model
model = Sequential()
model.add(Dense(input_dim=784, units=8, activation="tanh", kernel_initializer="normal"))
model.add(Dense(units=10, activation="softmax"))

model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.1), 
              metrics=['categorical_accuracy', 'mean_squared_error', 'mean_absolute_error'])

# Train
VERBOSE     = 0
EPOCHS      = 35

history = model.fit(X_train, y_train_binary, validation_data=(X_test, y_test_binary), epochs=EPOCHS, verbose=VERBOSE)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Evaluate original model

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

#print(history.history)
score = model.evaluate(X_test, y_test_binary, verbose=0)

print(f"Test loss:     {score[0]}") # loss is score 0 by definition?
print(f"Test accuracy: {score[1]}")
print(f"All scores in history: {score}")

Test loss:     0.542854045697621
Test accuracy: 0.8466666666666667
All scores in history: [0.542854045697621, 0.8466666666666667, 0.023199272668077833, 0.04052264713957196]


Using the original model designed for the moon data changed to take 784 inputs and have 10 outputs, the model have a loss of 0.543 and an accuracy score of 0.847. To optimize these results we tried another model.

### Building alternative model
For optimization we tried utilizing a convolutional neural network found: https://medium.com/@mjbhobe/mnist-digits-classification-with-keras-ed6c2374bd0e

This model uses a 2D convolutional neural network, so it require a different input (28, 28, 1) so we are going to reshape and normalize the data:

In [4]:
X, y = fetch_openml('mnist_784', cache=True, return_X_y=True)
X = X.reshape((X.shape[0], 28, 28, 1)).astype('float32')
X /= 255

# split dataset into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#one-hot encode classes
y_train_binary = to_categorical(y_train)
y_test_binary  = to_categorical(y_test)

assert y.ndim==1
assert y_train_binary.ndim==2
assert y_test_binary.ndim ==2

In [5]:
#Build and compile model
# https://medium.com/@mjbhobe/mnist-digits-classification-with-keras-ed6c2374bd0e
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


# Train
EPOCHS      = 15
history = model.fit(X_train, y_train_binary, validation_data=(X_test, y_test_binary),\
                    epochs=EPOCHS, batch_size=64)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 49000 samples, validate on 21000 samples
Epoch 1/15
49000/49000 [==============================] - 119s 2ms/step - loss: 0.2691 - categorical_accuracy: 0.9173 - val_loss: 0.0850 - val_categorical_accuracy: 0.9744
Epoch 2/15
49000/49000 [==============================] - 95s 2ms/step - loss: 0.0846 - categorical_accuracy: 0.9734 - val_loss: 0.0538 - val_categorical_accuracy: 0.9837
Epoch 3/15
49000/49000 [==============================] - 95s 2ms/step - loss: 0.0623 - categorical_accuracy: 0.9801 - val_loss: 0.0461 - val_categorical_accuracy: 0.9847
Epoch 4/15
49000/49000 [==============================] - 102s 2ms/step - loss: 0.0501 - categorical_accuracy: 0.9846 - val_loss: 0.0422 - val_categorical_accuracy: 0.9867
Epoch 5/15
49000/49000 [==============================] - 103s 2ms/step - loss: 0.0422 - categorical_accuracy: 0.9869 - val_loss: 0.0426 - val_categ

### Evaluate the alternative model

In [6]:
#print(history.history)
score = model.evaluate(X_test, y_test_binary, verbose=0)

print(f"Test loss:     {score[0]}") # loss is score 0 by definition?
print(f"Test accuracy: {score[1]}")
print(f"All scores in history: {score}")

Test loss:     0.042650930428374395
Test accuracy: 0.9875714285714285
All scores in history: [0.042650930428374395, 0.9875714285714285]


With a loss of 0.043 and accuracy 0.988, the convolutional nueral network performs better than the original configuration with a loss of  0.543 and an accuracy score of 0.847. We need to reshape our data, and the model takes a considerably long time to train, in this case 15 minutes.